**Instalación de Paquetes**

In [28]:
!pip install langdetect
from textblob import TextBlob
#import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import os
import nltk
import re
#import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=f9443f8a161380957c61c3c902afcf71eccd0a4c29eed6a251cd9d4e9943fea5
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [29]:
# Config de Pantalla
pd.set_option('display.max_colwidth', None)




**Autenticación de Credenciales**

In [43]:
# Mis credenciales API
consumerKey = input("consumerKey:")
consumerSecret = input("consumerKey:")
accessToken = input("consumerKey:")
accessTokenSecret = input("consumerKey:")


auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

consumerKey:fXbURnBEPcJCf7nPtjt0jicql
consumerKey:VTDQ4HAmSJXihXx3t3ZaCUxouYGT7G72BEXHUocj9XNFtaC7zI
consumerKey:1639307367161511945-YSdyxwFSo0ojbDnupZB2Ij4JXabjF6
consumerKey:edWSYoQXrmcVKqt9NSznpjLBHl2Bf6PcRVTEPLxOyNGve


**Obtención de tweets**

In [31]:
nltk.downloader.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)
keyword = input("Please enter keyword or hashtag to search:") #ingresar palabras claves o hashtags
noOfTweet = int(input ("Please enter how many tweets to analyze: ")) # ingresar el numero de tweets que se quieren analizar
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    polarity += analysis.sentiment.polarity
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1

    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, ".1f")
negative = format(negative, ".1f")
neutral = format(neutral, ".1f")

Please enter keyword or hashtag to search:#rodrigochaves,#presidencia,#informe1año
Please enter how many tweets to analyze: 300


Forbidden: ignored

In [ ]:
# Numero de Tweets (Total, Positivos,Negativos, Neutros)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

**Preparacón de Datos**


In [ ]:
# Limpieza
tweet_list.drop_duplicates(inplace = True)

# Limpieza de datos (RT, Puntuación etc)
# Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
# Removiendo RT, Puntuación etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)



In [ ]:
# Calculando valores Negativos, Positivos, Neutrales and Valores compuestos
tw_list[["polarity", "subjectivity"]] = tw_list["text"].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list["text"].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score["neg"]
    neu = score["neu"]
    pos = score["pos"]
    comp = score["compound"]
    if neg > pos:
        tw_list.loc[index, "sentiment"] = "negative"
    elif pos > neg:
        tw_list.loc[index, "sentiment"] = "positive"
    else:
        tw_list.loc[index, "sentiment"] = "neutral"
        tw_list.loc[index, "neg"] = neg
        tw_list.loc[index, "neu"] = neu
        tw_list.loc[index, "pos"] = pos
        tw_list.loc[index, "compound"] = comp
tw_list.head(10)

In [ ]:
#Función para crear nubes de palabras
def create_wordcloud(text):
    mask = np.array(Image.open("cloud.png"))
    #stopwords = set(STOPWORDS) # in english
    stopwords_sp = set(stopwords.words('spanish')) # in spanish
    wc = WordCloud(background_color="white",
    mask = mask,
    max_words=3000,
    stopwords=stopwords_sp,
    repeat=True)
    wc.generate(str(text))
    wc.to_file("wc.png")
    print("Word Cloud Saved Successfully")
    path="wc.png"
    display(Image.open(path))

In [ ]:
#Creando la nube de palabras
create_wordcloud(tw_list["text"].values)